This notebooks demonstrates basic usage of the fv3config package by doing two sequential 1-hour runs. It assumes that fv3config has been installed and that a docker image of fv3gfs-compiled-default is locally available.

In [1]:
import fv3config
import os
import shutil
import subprocess
from copy import deepcopy

In [2]:
model_image = 'fv3gfs-compiled-default'
archive = '/home/OliWM/.local/share/fv3gfs/archive'

In [3]:
def run_model(rundir, model_image, mounts):
    rundir_abs = os.path.abspath(rundir)
    rundir_mount = ['-v', f'{rundir_abs}:/FV3/rundir']
    run_command = ['docker', 'run', '--rm'] + rundir_mount + mounts + [model_image]
    fv3out = os.path.join(rundir, 'fv3out')
    fv3err = os.path.join(rundir, 'fv3err')
    with open(fv3out, 'w') as out, open(fv3err, 'w') as err:
        subprocess.call(run_command, stdout=out, stderr=err)

#### fv3config can provide a default configuration dict

In [4]:
initial_config = fv3config.get_default_config()
list(initial_config.keys())

['namelist',
 'diag_table',
 'data_table',
 'forcing',
 'initial_conditions',
 'experiment_name']

In [5]:
initial_config['namelist']['coupler_nml']

Namelist([('months', 0),
          ('days', 0),
          ('hours', 0),
          ('minutes', 30),
          ('seconds', 0),
          ('dt_atmos', 900),
          ('dt_ocean', 900),
          ('current_date', [2016, 8, 1, 0, 0, 0]),
          ('calendar', 'julian'),
          ('memuse_verbose', True),
          ('atmos_nthreads', 1),
          ('use_hyper_thread', True),
          ('ncores_per_node', 32)])

In [6]:
initial_config['diag_table']

'default'

Built-in options for diag_table are 'default', 'grid_spec' and 'no_output'. You can specify your own custom diag_table by using an absolute path:

In [7]:
initial_config['diag_table'] = os.path.abspath('diag_table_for_example_nb')

#### Change length of run to one hour

In [8]:
initial_config['namelist']['coupler_nml']['hours'] = 1
initial_config['namelist']['coupler_nml']['minutes'] = 0

#### Write run directory and copy in submit_job.sh script

In [9]:
initial_rundir = './initial_rundir'
fv3config.write_run_directory(initial_config, initial_rundir)
shutil.copy('submit_job.sh', os.path.join(initial_rundir, 'submit_job.sh'));

DataMissingError: Required data for running fv3gfs not available. Try python -m fv3config.download_data or ensure_data_is_downloaded()

In [10]:
fv3config.ensure_data_is_downloaded()
fv3config.write_run_directory(initial_config, initial_rundir)
shutil.copy('submit_job.sh', os.path.join(initial_rundir, 'submit_job.sh'));

#### Run model

Because fv3config uses symlinks when writing a run directory, it is currently necessary to bind-mount the directory where fv3config stores its data if running from outside a docker container.

In [11]:
archive_mount = ['-v', f'{archive}:{archive}']
run_model(initial_rundir, model_image, archive_mount)

#### Configure a run to initialize from end of previous run

In [12]:
ic_directory = os.path.abspath(os.path.join(initial_rundir, 'RESTART'))
restart_config = deepcopy(initial_config)
restart_config['initial_conditions'] = ic_directory
restart_config['namelist']['fv_core_nml']['warm_start'] = True
restart_config['namelist']['fv_core_nml']['external_ic'] = False
restart_config['namelist']['fv_core_nml']['nggps_ic'] = False
restart_config['namelist']['fv_core_nml']['make_nh'] = False
restart_config['namelist']['fv_core_nml']['mountain'] = True
restart_config['namelist']['fv_core_nml']['na_init'] = 0

restart_rundir = './restart_rundir'
fv3config.write_run_directory(restart_config, restart_rundir)
shutil.copy('submit_job.sh', os.path.join(restart_rundir, 'submit_job.sh'));

Again, because the initial conditions are symlinked, it is necessary to bind-mount the RESTART directory from the initial run:

In [13]:
ic_mount = ['-v', f'{ic_directory}:{ic_directory}']
run_model(initial_rundir, model_image, archive_mount + ic_mount)